In [41]:
import json
import numpy as np

In [161]:
with open('test.json') as file:
    data = json.load(file)

match data['goal']:
    case 'min':
        f = -1 * np.array(data['f'])
    case 'max':
        f = np.array(data['f'])

n = len(data['constraints'][0]['coefs'])
m = len(data['constraints'])
A = np.zeros((0, n))
b = np.empty((0, 1))
for constraint in data['constraints']:
    coefs = np.array(constraint['coefs'])
    match constraint['type']:
        case 'eq':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
        case 'lte':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            coefs = np.append(coefs, 1)
            A = np.hstack([A, np.zeros((A.shape[0], 1))])
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
        case 'gte':
            if len(coefs) != A.shape[1]:
                coefs = np.pad(coefs, (0, A.shape[1] - len(coefs)), 'constant', constant_values=0)
            coefs = np.append(coefs, -1)
            A = np.hstack([A, np.zeros((A.shape[0], 1))])
            A = np.vstack([A, coefs])
            b = np.append(b, constraint['b'])
f = np.pad(f, (0, A.shape[1] - len(f)), 'constant', constant_values=0) if A.shape[1] > len(f) else f
f, A, b

(array([2, 3, 0, 0, 0, 0]),
 array([[1., 0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0.],
        [1., 1., 0., 0., 1., 0.],
        [1., 2., 0., 0., 0., 1.]]),
 array([40., 30., 60., 80.]))

In [147]:
A_ = np.array([[-2, 1, 1, 0, 0],
             [-1, 2, 0, 1, 0],
             [1, 0, 0, 0, 1]])

b_ = np.array([2, 7, 3])

c_ = np.array([-1, -2, 0, 0, 0])

In [178]:

def to_tableau(c, A, b):
    xb = [eq + [x] for eq, x in zip(A, b)]
    z = c + [0]
    return xb + [z]

def can_be_improved(tableau):
    z = tableau[-1]
    return any(x > 0 for x in z[:-1])

import math

def get_pivot_position(tableau):
    z = tableau[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0)
    
    restrictions = []
    for eq in tableau[:-1]:
        el = eq[column]
        restrictions.append(math.inf if el <= 0 else eq[-1] / el)

    row = restrictions.index(min(restrictions))
    return row, column

def pivot_step(tableau, pivot_position):
    new_tableau = [[] for eq in tableau]
    
    i, j = pivot_position
    pivot_value = tableau[i][j]
    new_tableau[i] = np.array(tableau[i]) / pivot_value
    
    for eq_i, eq in enumerate(tableau):
        if eq_i != i:
            multiplier = np.array(new_tableau[i]) * tableau[eq_i][j]
            new_tableau[eq_i] = np.array(tableau[eq_i]) - multiplier
   
    return new_tableau

def is_basic(column):
    return sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1

def get_solution(tableau):
    columns = np.array(tableau).T
    solutions = []
    for column in columns:
        solution = 0
        if is_basic(column):
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)
        
    return solutions

def simplex(c, A, b):
    tableau = to_tableau(c, A, b)

    while can_be_improved(tableau):
        pivot_position = get_pivot_position(tableau)
        tableau = pivot_step(tableau, pivot_position)

    return get_solution(tableau)

solution = simplex(f.tolist(), A.tolist(), b)
solution

[40.0, 20.0, 0.0, 10.0, 0, 0, 0]